In [2]:
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [3]:
def order_book(month,day):
    data = []
    datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/High_Frequency_Trading_Strategy_using_machine_learning/Limit_Order_Book_Best_3/order_book_3_2014'\
                + '_' + str(month) + '_' + str(day) + '.csv'
    order_book = pd.read_csv(datapath,sep=',')
    bid_price_1 = np.array(map(float,order_book['Bid'][1::4]))/100.0
    bid_price_2 = np.array(map(float,order_book['Bid'][2::4]))/100.0
    bid_price_3 = np.array(map(float,order_book['Bid'][3::4]))/100.0
    timestamp = np.array(order_book['Bid_Quantity'][0::4])
    bid_quantity_1 = np.array(map(float,order_book['Bid_Quantity'][1::4]))
    bid_quantity_2 = np.array(map(float,order_book['Bid_Quantity'][2::4]))
    bid_quantity_3 = np.array(map(float,order_book['Bid_Quantity'][3::4]))
    ask_price_1 = np.array(map(float,order_book['Ask'][1::4]))/100.0
    ask_price_2 = np.array(map(float,order_book['Ask'][2::4]))/100.0
    ask_price_3 = np.array(map(float,order_book['Ask'][3::4]))/100.0
    ask_quantity_1 = np.array(map(float,order_book['Ask_Quantity'][1::4]))
    ask_quantity_2 = np.array(map(float,order_book['Ask_Quantity'][2::4]))
    ask_quantity_3 = np.array(map(float,order_book['Ask_Quantity'][3::4]))
    
    bid_quantity_1[isnan(bid_quantity_1)] = 0
    bid_quantity_2[isnan(bid_quantity_2)] = 0
    bid_quantity_3[isnan(bid_quantity_3)] = 0
    ask_quantity_1[isnan(ask_quantity_1)] = 0
    ask_quantity_2[isnan(ask_quantity_2)] = 0
    ask_quantity_3[isnan(ask_quantity_3)] = 0
    
    return timestamp,order_book,bid_price_1,bid_price_2,bid_price_3,bid_quantity_1,\
            bid_quantity_2,bid_quantity_3,ask_price_1,ask_price_2,ask_price_3,ask_quantity_1,\
            ask_quantity_2,ask_quantity_3

In [4]:
def time_transform(timestamp_time):
    time_second_basic = []
    time_second = []
    for i in range(0,len(timestamp_time),1):
        second = float(timestamp_time[i][11])*36000 + float(timestamp_time[i][12])*3600+\
                    float(timestamp_time[i][14])*600 + float(timestamp_time[i][15])*60+\
                    float(timestamp_time[i][17])*10 + float(timestamp_time[i][18])  
        time_second_basic.append(second - 32400.0)
        time_second.append(second)
    return np.array(time_second),np.array(time_second_basic)

In [129]:
def rise_ask(Ask1,timestamp_time_second,before_time):
    Ask1[Ask1 == 0] = mean(Ask1)
    rise_ratio = []
    index = np.where(timestamp_time_second >= before_time)[0][0]    
    #open first before_time mins
    for i in range(0,index,1):
        rise_ratio_ = round((Ask1[i] - Ask1[0])*(1.0)/Ask1[0]*100,5)
        rise_ratio.append(rise_ratio_)
    for i in range(index,len(Ask1),1):
        index_start = np.where(timestamp_time_second[:i] >= timestamp_time_second[i] - before_time)[0][0]
        rise_ratio_ = round((Ask1[i] - Ask1[index_start])*(1.0)/Ask1[index_start]*100,5)
        rise_ratio.append(rise_ratio_)
    return np.array(rise_ratio)

In [7]:
def weight_pecentage(w1,w2,w3):
    Weight_Ask = (w1 * ask_quantity_1 + w2 * ask_quantity_2 + w3 * ask_quantity_3)
    Weight_Bid = (w1 * bid_quantity_1 + w2 * bid_quantity_2 + w3 * bid_quantity_3)
    W_AB = Weight_Ask/Weight_Bid
    W_A_B = (Weight_Ask - Weight_Bid)/(Weight_Ask + Weight_Bid)
    return W_AB,W_A_B

In [8]:
month = 1
day = 2
timestamp,order_book,bid_price_1,bid_price_2,bid_price_3,\
bid_quantity_1,bid_quantity_2,bid_quantity_3,\
ask_price_1,ask_price_2,ask_price_3,ask_quantity_1,\
ask_quantity_2,ask_quantity_3 = order_book(month,day)

In [9]:
time_second,time_second_basic = time_transform(timestamp)

In [11]:
traded_time = 300
# 09:00 ~ 09:15
time1 = 0
time2 = 900
traded_micsecond_1,index_1 = traded_label_micsecond(time1,time2,time_second_basic,\
                                                    bid_price_1,ask_price_1,traded_time)            
# 09:15 ~ 11:30
time1 = 900+1
time2 = 9000
traded_micsecond_2,index_2 = traded_label_micsecond(time1,time2,time_second_basic,\
                                                    bid_price_1,ask_price_1,traded_time)            
# 11:30 ~ 1:00
time1 = 9000+1
time2 = 14400
traded_micsecond_3,index_3 = traded_label_micsecond(time1,time2,time_second_basic,\
                                                    bid_price_1,ask_price_1,traded_time)            
# 01:00 ~ 16:00
time1 = 14400+1
time2 = 25200
traded_micsecond_4,index_4 = traded_label_micsecond(time1,time2,time_second_basic,\
                                                    bid_price_1,ask_price_1,traded_time)

In [186]:
import time 
start = time.time()

before_time = 60.0 * 5 #second
index = np.where(time_second_basic <= 0.0)[0][-1]   
Ask1 = ask_price_1[np.where(time_second_basic <= 0.0)[0][-1]:]
rise_ratio_ask = rise_ask(Ask1, time_second_basic, before_time)

#Weight Depth
w1,w2,w3 = [50.0,30.0,20.0]
W_AB,W_A_B = weight_pecentage(w1,w2,w3)

end = time.time()  
print "Total time = %f"%(end - start)

Total time = 1.374204


In [243]:
def traded_label_one_second(time1, time2, time_second_basic,\
                            bid_price_1, ask_price_1, traded_time,\
                            rise_ratio_ask,W_AB,W_A_B):
    traded = []
    index_ = []
    rise_ratio_second = []
    weight_1 = []
    weight_2 = []
    index_1 = np.where(time_second_basic <= 0)[0][-1]
    for i in range(time1, time2, 1):
        index = np.where(time_second_basic <= i)[0][-1]

        if i == time1:
            index_.append(index)
        if i == time2 - 1:
            index_.append(index)
            
        if i < 25200 - traded_time:
            index_min = np.where(time_second_basic <= i + traded_time)[0][-1]
            if bid_price_1[index] > min(ask_price_1[index:index_min]):
                traded.append(1)
            else:
                traded.append(0)
        elif i >= 25200 - traded_time:
            if bid_price_1[index] > ask_price_1[-1]:
                traded.append(1)
            else:
                traded.append(0)
        rise_ratio_second.append(rise_ratio_ask[index - index_1])
        weight_1.append(W_AB[index - index_1])
        weight_2.append(W_A_B[index - index_1])
    return traded, index_, rise_ratio_second,weight_1,weight_2

In [292]:
# 09:00 ~ 09:15
time1 = 0
time2 = 900
traded_time = 900
traded_1 ,index_1,rise_ratio_second1,WD_11,WD_12 = \
    traded_label_one_second(time1,time2,time_second_basic,bid_price_1,\
                            ask_price_1,traded_time,rise_ratio_ask,W_AB,W_A_B)
    
len(np.where(np.array(traded_1) == 0)[0]),len(np.where(np.array(traded_1) == 1)[0]),len(traded_1),index_1

(2, 898, 900, [294, 1008])

In [293]:
# 09:00 ~ 09:15
time1 = 900
time2 = 9000
traded_time = 900
traded_2 ,index_2, rise_ratio_second2,WD_21,WD_22 = \
    traded_label_one_second(time1,time2,time_second_basic,bid_price_1,\
                            ask_price_1,traded_time,rise_ratio_ask,W_AB,W_A_B)

len(np.where(np.array(traded_2) == 0)[0]),len(np.where(np.array(traded_2) == 1)[0]),len(traded_2),index_2

(1811, 6289, 8100, [1021, 29797])

In [294]:
# 09:00 ~ 09:15
time1 = 9000
time2 = 14400
traded_time = 900
traded_3 ,index_3, rise_ratio_second3,WD_31,WD_32 = \
    traded_label_one_second(time1,time2,time_second_basic,bid_price_1,\
                            ask_price_1,traded_time,rise_ratio_ask,W_AB,W_A_B)
    
len(np.where(np.array(traded_3) == 0)[0]),len(np.where(np.array(traded_3) == 1)[0]),len(traded_3),index_

(5400, 0, 5400, [29798, 31268])

In [295]:
# 12:00 ~ 16:00
time1 = 14400
time2 = 25200
traded_time = 900
traded_4 ,index_4, rise_ratio_second4, WD_41, WD_42 = \
    traded_label_one_second(time1,time2,time_second_basic,bid_price_1,ask_price_1,\
                            traded_time,rise_ratio_ask,W_AB,W_A_B)
    
len(np.where(np.array(traded_4) == 0)[0]),len(np.where(np.array(traded_4) == 1)[0]),len(traded_4),index_4

(7076, 3724, 10800, [31276, 46681])

In [313]:
data.shape

(19800, 3)

In [316]:
data = np.array([traded_1 + traded_2 + traded_4,\
                rise_ratio_second1 + rise_ratio_second2 + rise_ratio_second4,\
                WD_11 + WD_21 + WD_41,WD_12 + WD_22 + WD_42]).T

In [317]:
pd.DataFrame(data, columns = ['label', 'rise', 'depth_divid', 'depth_diff'])

,label,rise,depth_divid,depth_diff
0,0.0,0.00000,0.833333,-0.090909
1,0.0,0.00000,0.833333,-0.090909
2,1.0,0.21097,1.002838,0.001417
3,1.0,0.21097,1.004739,0.002364
4,1.0,0.21097,1.004739,0.002364
5,1.0,0.21097,1.010427,0.005186
6,1.0,0.21097,1.010427,0.005186
7,1.0,0.21097,1.008515,0.004239
8,1.0,0.21097,1.008515,0.004239
9,1.0,0.21097,1.008515,0.004239
